## Importing Required Libraries and dependencies.




In [77]:
#Import nessesary modules
import yfinance as yf
import pandas as pd
import json 
import numpy as np
import sqlite3
import streamlit as st


ModuleNotFoundError: No module named 'streamlit'

## Step 1: Propt the user to customize choice for analysis

### 1.1 Prompt the user about how much capital they wish to invest??? Is it needed or we can calculate mean() for each dollar invested ?
### !!! Create widget on Streamlit for the prompt!!!

In [78]:
#Ask user how much is the allocation for Equity Allocation
# Check the input 
while True:
    try:
        equity_allocation = float(input('Please enter the amount of capital you wish to invest :'))
        break
    except:
        print('Please enter a numerical value.')


Please enter the amount of capital you wish to invest : 1000


## 1.2 Prompt the user how many years of historical data they wish to pull for the analysis.
### !!! Create widget on Streamlit for the prompt!!!

In [79]:
# Prompt the user how many years of historical data they wish to pull for the analysis
# Check the input 
while True:
    try:
        num_years = int(input('Please enter the number of years (1-10) of historical data that you wish to retrieve for the analysis :'))
        break
    except:
        print('Please enter a numerical value.')


Please enter the number of years (1-10) of historical data that you wish to retrieve for the analysis : 10


## 1.3 Prompt the user to select assets from each asset class.
### ??? Does the number of assets in each asset class should be the same for correct analysis???

### !!! Create widget on Streamlit for the prompt!!!

## Step 2 : 
### Data Collection and Cleaning
The first step is to collect data for the 3 different assets - crypto, commodities, and stocks. Using **yfinance** library for Pandas pull data from the Yahoo finance API. The data pulled for each asset class would have 10 tickers. Store the data for each asset class in separate Pandas DataFrames.

In [80]:
# Define the lists of asset classes and their respective tickers. Set it as variables, so the selection can be changed easier if neded. 
### VARIATION 1
# crypto = ['BTC-USD', 'ETH-USD', 'DOGE-USD', 'MATIC-USD', 'AVAX-USD', 'SOL-USD', 'TRX-USD', 'ATOM-USD', 'UNI7083-USD', 'LINK-USD']
# stocks = ['AMZN', 'AAPL', 'TSLA', 'GOOGL', 'NVDA', 'MSFT', 'TSM', 'META','XOM', 'LAC']
# commodities = ['GC=F', 'SI=F', 'CL=F', 'HG=F', 'LBS=F', 'ZS=F', 'GF=F', 'KE=F', 'CT=F', 'ZR=F']

# # Collect data for crypto asset class
# crypto_df = pd.DataFrame()
# for ticker in crypto:
#     crypto_df[ticker] = yf.download(ticker)['Close']

# # Collect data for stocks asset class
# stocks_df = pd.DataFrame()
# for ticker in stocks:
#     stocks_df[ticker] = yf.download(ticker)['Close']

# # Collect data for commodities asset class
# commodities_df = pd.DataFrame()
# for ticker in commodities:
#     commodities_df[ticker] = yf.download(ticker)['Close']


In [81]:
# crypto_df.head(200)
# The problem with crypto historic data is taht diferent start times of the data(begining of the trading period is diffferent for each crypto)  

In [82]:
### VARIATION 2
# Define the a dictionary of lists for each asset class
# api_pull = {'crypto' : ['BTC-USD', 'ETH-USD', 'DOGE-USD', 'MATIC-USD', 'AVAX-USD', 'SOL-USD', 'TRX-USD', 'ATOM-USD', 'UNI7083-USD', 'LINK-USD']
#             ,'stocks' : ['AMZN', 'AAPL', 'TSLA', 'GOOGL', 'NVDA', 'MSFT', 'TSM', 'META','XOM', 'LAC']
#             ,'commodities' : ['GC=F', 'SI=F', 'CL=F', 'HG=F', 'LBS=F', 'ZS=F', 'GF=F', 'KE=F', 'CT=F', 'ZR=F']
           
#            }

# # Store data for each asset class in a dictionary
# data = {}
# for asset_class in api_pull:
#     for ticker in api_pull[asset_class]:
#         data[asset] = pd.DataFrame(yf.download(asset, start='2010-01-10', end ='2023-01-01'))

            

In [83]:
# VARIATION 3
# Define the lists of asset classes and their respective tickers. Set it as variables, so the selection can be changed easier if neded.
api_pull = {'crypto' : ['BTC-USD', 'ETH-USD', 'DOGE-USD', 'MATIC-USD', 'AVAX-USD', 'SOL-USD', 'TRX-USD', 'ATOM-USD', 'UNI7083-USD', 'LINK-USD'],
            'stocks' : ['AMZN', 'AAPL', 'TSLA', 'GOOGL', 'NVDA', 'MSFT', 'TSM', 'META','XOM', 'LAC'],
            'commodities' : ['GC=F', 'SI=F', 'CL=F', 'HG=F', 'LBS=F', 'ZS=F', 'GF=F', 'KE=F', 'CT=F', 'ZR=F']}

# Store data for each asset class in a dictionary
# Use .items()to iterate over the key-value pairs of the api_pull dictionary
# pull data for the number of years requested by the user
data = {}
for asset_class, tickers in api_pull.items():
    data[asset_class] = {}
    for ticker in tickers:
        data[asset_class][ticker] = yf.Ticker(ticker).history(period=f"{num_years}y")


In [85]:
# Review pulled data:
data 

{'crypto': {'BTC-USD':                                    Open          High           Low  \
  Date                                                                  
  2014-09-17 00:00:00+00:00    465.864014    468.174011    452.421997   
  2014-09-18 00:00:00+00:00    456.859985    456.859985    413.104004   
  2014-09-19 00:00:00+00:00    424.102997    427.834991    384.532013   
  2014-09-20 00:00:00+00:00    394.673004    423.295990    389.882996   
  2014-09-21 00:00:00+00:00    408.084991    412.425995    393.181000   
  ...                                 ...           ...           ...   
  2023-02-04 00:00:00+00:00  23446.320312  23556.949219  23291.794922   
  2023-02-05 00:00:00+00:00  23332.248047  23423.435547  22841.759766   
  2023-02-06 00:00:00+00:00  22954.021484  23119.279297  22692.025391   
  2023-02-07 00:00:00+00:00  22757.267578  23310.974609  22756.257812   
  2023-02-09 00:00:00+00:00  23267.400391  23351.509766  22737.466797   
  
                           

In [86]:
# Create a data frame for each asset class with the close column
# Access the 'Close' column of the data for each asset class
# Extract the values of the data dictionary and then access the 'Close' column of each dataframe.
df_crypto = pd.DataFrame({ticker: data['crypto'][ticker]['Close'] for ticker in api_pull['crypto']})
df_stocks = pd.DataFrame({ticker: data['stocks'][ticker]['Close'] for ticker in api_pull['stocks']})
df_commodities = pd.DataFrame({ticker: data['commodities'][ticker]['Close'] for ticker in api_pull['commodities']})


In [87]:
# Replace all the NaN values in a data frame using the fillna() method.
df_crypto.fillna(0, inplace=True)
df_stocks.fillna(0, inplace=True)
df_commodities.fillna(0, inplace=True)

In [88]:
# Review first and last 10 rows for crypto assets class
display(df_crypto.head(10))
df_crypto.tail(10)



,BTC-USD,ETH-USD,DOGE-USD,MATIC-USD,AVAX-USD,SOL-USD,TRX-USD,ATOM-USD,UNI7083-USD,LINK-USD
Date,,,,,,,,,,
2014-09-17 00:00:00+00:00,457.334015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-09-18 00:00:00+00:00,424.440002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-09-19 00:00:00+00:00,394.795990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-09-20 00:00:00+00:00,408.903992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-09-21 00:00:00+00:00,398.821014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-09-22 00:00:00+00:00,402.152008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-09-23 00:00:00+00:00,435.790985,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-09-24 00:00:00+00:00,423.204987,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-09-25 00:00:00+00:00,411.574005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,BTC-USD,ETH-USD,DOGE-USD,MATIC-USD,AVAX-USD,SOL-USD,TRX-USD,ATOM-USD,UNI7083-USD,LINK-USD
Date,,,,,,,,,,
2023-01-30 00:00:00+00:00,22840.138672,1567.326538,0.088337,1.087235,19.739170,23.946321,0.062247,13.056948,6.508925,6.906292
2023-01-31 00:00:00+00:00,23139.283203,1586.535400,0.096167,1.111333,19.844500,23.952547,0.062220,13.379325,6.568031,6.952308
2023-02-01 00:00:00+00:00,23723.769531,1641.792725,0.094171,1.194967,20.965364,24.993359,0.063193,14.868429,6.812751,7.216624
2023-02-02 00:00:00+00:00,23471.871094,1643.241577,0.090979,1.180444,21.180059,24.265059,0.063761,14.363241,7.102803,7.102140
2023-02-03 00:00:00+00:00,23449.322266,1664.745605,0.092987,1.244227,21.365187,24.680897,0.064122,14.996407,7.071822,7.290623
2023-02-04 00:00:00+00:00,23331.847656,1667.059204,0.095782,1.250628,21.024635,24.317745,0.063824,14.679261,7.204175,7.238031
2023-02-05 00:00:00+00:00,22955.666016,1631.645874,0.092095,1.204501,20.108017,23.493439,0.064085,14.754556,6.899310,6.984616
2023-02-06 00:00:00+00:00,22760.109375,1616.247070,0.089549,1.190415,19.585035,22.706388,0.063194,14.142922,6.652495,6.832168
2023-02-07 00:00:00+00:00,23264.291016,1672.003540,0.092576,1.268975,20.845219,23.861357,0.065774,14.957318,6.984529,7.198653


In [89]:
# Review first and last 10 rows for stocks assets class
display(df_stocks.head(10))
df_stocks.tail(10)

,AMZN,AAPL,TSLA,GOOGL,NVDA,MSFT,TSM,META,XOM,LAC
Date,,,,,,,,,,
2013-02-11 00:00:00-05:00,12.8605,14.826402,2.561333,19.580080,2.888787,22.972578,13.752826,28.260000,58.171101,1.10
2013-02-12 00:00:00-05:00,12.9350,14.454766,2.526000,19.537037,2.870314,22.989077,13.767714,27.370001,58.289700,1.10
2013-02-13 00:00:00-05:00,13.4735,14.427270,2.563333,19.591091,2.856460,23.112764,13.700741,27.910000,58.428082,1.00
2013-02-14 00:00:00-05:00,13.4620,14.414298,2.551333,19.715216,2.939589,23.121008,13.819804,28.500000,58.329243,0.90
2013-02-15 00:00:00-05:00,13.2545,14.215655,2.469333,19.842093,2.939589,23.096268,13.790037,28.320000,58.223793,0.90
2013-02-19 00:00:00-05:00,13.4875,14.210404,2.618667,20.191441,2.900334,23.320747,14.251444,28.930000,58.856392,0.90
2013-02-20 00:00:00-05:00,13.3205,13.866258,2.569333,19.831331,2.858768,23.171093,14.057950,28.459999,58.625774,0.85
2013-02-21 00:00:00-05:00,13.2970,13.780066,2.344000,19.908157,2.837986,22.855164,13.909110,27.280001,58.375389,0.90
2013-02-22 00:00:00-05:00,13.2710,13.926803,2.407333,20.012762,2.891098,23.079639,14.065394,27.129999,58.777328,0.85


,AMZN,AAPL,TSLA,GOOGL,NVDA,MSFT,TSM,META,XOM,LAC
Date,,,,,,,,,,
2023-01-26 00:00:00-05:00,99.220001,143.960007,160.270004,97.519997,198.020004,248.000000,93.449997,147.300003,117.760002,22.100000
2023-01-27 00:00:00-05:00,102.239998,145.929993,177.899994,99.370003,203.649994,248.160004,93.300003,151.740005,115.610001,22.500000
2023-01-30 00:00:00-05:00,100.550003,143.000000,166.660004,96.940002,191.619995,242.710007,93.129997,147.059998,113.559998,21.990000
2023-01-31 00:00:00-05:00,103.129997,144.289993,173.220001,98.839996,195.369995,247.809998,92.730003,148.970001,116.010002,25.230000
2023-02-01 00:00:00-05:00,105.150002,145.429993,181.410004,100.430000,209.429993,252.750000,94.739998,153.119995,114.739998,24.700001
2023-02-02 00:00:00-05:00,112.910004,150.820007,188.270004,107.739998,217.089996,264.600006,96.650002,188.770004,111.150002,24.959999
2023-02-03 00:00:00-05:00,103.389999,154.500000,189.979996,104.779999,211.000000,258.350006,94.660004,186.529999,111.919998,24.320000
2023-02-06 00:00:00-05:00,102.180000,151.729996,194.759995,102.900002,210.889999,256.769989,91.830002,186.059998,111.730003,23.459999
2023-02-07 00:00:00-05:00,102.110001,154.649994,196.809998,107.639999,221.729996,267.559998,94.550003,191.619995,114.919998,25.889999


In [90]:
# Review first and last 10 rows for commodities assets class
display(df_commodities.head(10))
df_commodities.tail(10)

,GC=F,SI=F,CL=F,HG=F,LBS=F,ZS=F,GF=F,KE=F,CT=F,ZR=F
Date,,,,,,,,,,
2013-02-08 00:00:00-05:00,1666.000000,31.424999,95.720001,3.7535,387.799988,1452.50,145.000000,799.75,82.669998,1634.5
2013-02-11 00:00:00-05:00,1648.199951,30.895000,97.029999,3.7180,384.299988,1431.50,144.800003,788.75,82.919998,1584.5
2013-02-12 00:00:00-05:00,1648.699951,31.004000,97.510002,3.7365,389.299988,1420.75,143.250000,778.00,81.809998,1566.0
2013-02-13 00:00:00-05:00,1644.199951,30.854000,97.010002,3.7340,393.899994,1423.00,141.350006,780.25,80.820000,1585.5
2013-02-14 00:00:00-05:00,1634.699951,30.343000,97.309998,3.7300,395.500000,1418.00,142.675003,775.00,81.019997,1572.0
2013-02-15 00:00:00-05:00,1608.800049,29.839001,95.860001,3.7310,399.799988,1424.50,143.375000,777.50,81.320000,1583.5
2013-02-19 00:00:00-05:00,1603.599976,29.413000,96.660004,3.6445,389.799988,1470.25,143.175003,769.00,82.180000,1592.0
2013-02-20 00:00:00-05:00,1577.599976,28.615000,94.459999,3.6055,383.200012,1482.75,140.725006,777.25,82.279999,1602.5
2013-02-21 00:00:00-05:00,1578.199951,28.695000,92.839996,3.5505,378.000000,1487.75,140.699997,757.25,81.309998,1583.5


,GC=F,SI=F,CL=F,HG=F,LBS=F,ZS=F,GF=F,KE=F,CT=F,ZR=F
Date,,,,,,,,,,
2023-01-26 00:00:00-05:00,1929.099976,23.930000,81.010002,4.2670,488.200012,1523.50,179.574997,864.75,87.500000,1831.00
2023-01-27 00:00:00-05:00,1928.599976,23.527000,79.680000,4.2240,492.500000,1509.50,183.475006,869.25,86.889999,1831.00
2023-01-30 00:00:00-05:00,1922.900024,23.645000,77.900002,4.2035,516.500000,1535.25,183.925003,873.75,85.099998,1812.00
2023-01-31 00:00:00-05:00,1929.500000,23.747999,78.870003,4.2285,524.099976,1538.00,186.149994,878.75,86.220001,1782.50
2023-02-01 00:00:00-05:00,1927.800049,23.521000,76.410004,4.1110,509.600006,1520.25,183.250000,884.25,85.610001,1784.50
2023-02-02 00:00:00-05:00,1916.300049,23.534000,75.879997,4.0940,512.200012,1534.25,185.925003,880.75,86.389999,1778.00
2023-02-03 00:00:00-05:00,1862.900024,22.334000,73.389999,4.0630,499.799988,1532.00,186.100006,873.00,85.430000,1774.00
2023-02-06 00:00:00-05:00,1866.199951,22.177000,74.110001,4.0435,486.700012,1521.25,187.699997,876.00,83.269997,1754.50
2023-02-07 00:00:00-05:00,1871.699951,22.117001,77.139999,4.0885,469.500000,1515.25,187.199997,885.75,85.629997,1747.50


## Step 3 : 
### Prepare data ready for the Monte Carlo simulation
### ??? Should we prompt user to enter the number of simulations for the model ???
### !!! Keep in mind that for 'stocks' and 'comodities' trading_days = 252, but for 'crypto' trading_days = 365.!!!

In [92]:
## Prepare the data for Monte Carlo simulation
returns = portfolio_data.pct_change() #Calculate returns for each asset
mean_return = returns.mean()
cov_matrix = returns.cov()

NameError: name 'portfolio_data' is not defined

In [ ]:
# Set the number of simulations and trading days
num_simulations = 1000
stocks_n_comodities_num_trading_days = 252
crypto_num_trading_days = 365

## Step 4 : 
### Plot an overlay of the closing prices for each asset class (10 assetset each)
### ??? Plot the returns ???


KeyError: 'Close'

,Open,High,Low,Close,Adj Close,Volume,close_returns
Date,,,,,,,
2010-01-15,1407.0,1410.0,1395.0,1397.5,1397.5,487,0.019329
2010-01-19,1399.5,1413.5,1370.0,1392.0,1392.0,745,-0.003936
2010-01-20,1396.0,1399.0,1345.5,1349.0,1349.0,1565,-0.030891
2010-01-21,1349.0,1388.5,1340.5,1388.0,1388.0,1738,0.028910
2010-01-22,1384.0,1424.5,1366.5,1417.0,1417.0,1023,0.020893


,Open,High,Low,Close,Adj Close,Volume,close_returns,return
Date,,,,,,,,
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700,-0.069843,NaN
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600,0.035735,0.035735
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100,-0.024659,-0.024659
2014-09-22,399.100006,406.915985,397.130005,402.152008,402.152008,24127600,0.008352,0.008352
2014-09-23,402.092010,441.557007,396.196991,435.790985,435.790985,45099500,0.083647,0.083647
...,...,...,...,...,...,...,...,...
2022-12-27,16919.291016,16959.845703,16642.072266,16717.173828,16717.173828,15748580239,-0.011976,-0.011976
2022-12-28,16716.400391,16768.169922,16497.556641,16552.572266,16552.572266,17005713920,-0.009846,-0.009846
2022-12-29,16552.322266,16651.755859,16508.683594,16642.341797,16642.341797,14472237479,0.005423,0.005423


In [93]:

# Limit the number of securities for each alloocation( depends on the data fed)

# Get combination of alocated assets and weighting 
# Use KMeans algorithm for clustering 
#from sklearn.cluster import KMeans
#import numpy as np






In [94]:
#Data from 3 assets with their weight amounts
#Limit each equity t0 allocate 10 items 
assets = {'equity':range(0,equity_alloc,10),'commodity':range(0,commodity_alloc,10),'crypto':range(0,crypto_alloc,10)}




NameError: name 'equity_alloc' is not defined

In [6]:
#Allowing only 3 cluster of assets 
kmeans = KMeans(n_clusters=3)



In [7]:
#Fit the model to the asset data 
kmeans.fit(assets)



TypeError: float() argument must be a string or a number, not 'dict'

In [ ]:
# Plot vizualization fort the clusters 

#### For streamlit install pip install streamlit==0.62.0
### there is a problem problem was related to Python version.that pyarrow is a requirement in streamlit version 0.63.0. So you can use pip install streamlit==0.62.0 and it will work just fine